<h1>Project -  Stock Market Prediction</h1>


##Question 1: Pre-processing &  data exploration

In [27]:
import nltk
#nltk.download()
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import sklearn.datasets as sk
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [28]:
DataDirectoryPath= "C:\\Users\\shachar wild\\PycharmProjects\\Project-Stock-Market-Prediction\\Data"

In [29]:
stopWords=(stopwords.words('english'))

In [30]:
# function that clean the text
def cleanText(text):
    text = str(text)
    if text.startswith("b\"") or text.startswith("b\'"):
        text = text[1:]
    text = text.replace("\"", "").replace("\'", "")
    # lower case
    text = text.lower()
    # tokenize
    filtered_sentence = word_tokenize(text)
    # # remove stop words
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    # stemming
    ps = PorterStemmer()
    for i in range(len(filtered_sentence)-1):
        try:
            filtered_sentence[i] = ps.stem(filtered_sentence[i])
        except Exception as inst:
            filtered_sentence[i]=filtered_sentence[i]
    # remove stop words after stem
    filtered_sentence = [w for w in filtered_sentence if not w in stopWords]
    filtered_sentence = [w for w in filtered_sentence if len(w)>1]
    text= " ".join(filtered_sentence)
    return text.strip()

In [31]:
# read the data into data frame
combinedDataDF = pd.read_csv( DataDirectoryPath+"\\Combined_News_DJIA.csv")

In [35]:
# clean the text (stopWords, tokenize, stemming)
headlines = combinedDataDF.columns[range(2, 27)]
combinedDataDF[headlines] = combinedDataDF[headlines].applymap(cleanText)

Class distribution:

In [36]:
classDist = []
for c in [0, 1]:
    temp=[c, combinedDataDF["Label"].value_counts()[c]]
    classDist.append(temp)
pd.DataFrame(classDist, columns=['Class', 'number of Samples'])

,Class,number of Samples
0,0,924
1,1,1065


Terms frequency:

In [37]:
# get 15 most common words for each class
termsFreqPerClass=[]
print ("Terms frequency - for each class, number of appearances of term:")
for currClass in [0, 1]:
    # aspect 1
    class_df = combinedDataDF.loc[combinedDataDF["Label"] == currClass]
    tempDF=pd.DataFrame()
    tempDF['text']=class_df[class_df.columns[2:]].apply(
        lambda x: " ".join(x.dropna()),
        axis=1
    )
    temp=[" ".join(set(w.split(" "))) for w in tempDF['text']] #remove duplicates
    temp=" ".join(temp)
    termFreq=pd.Series((temp).split()).value_counts()
    termsFreqDF=pd.DataFrame()
    termsFreqDF['Term']=termFreq.index
    termsFreqDF['Freq']=list(termFreq)
    setFreq=set(termsFreqDF['Freq'])
    # get top 12
    termFreq=termFreq[:12]
    zipedFreq=zip(termFreq.index,termFreq)
    zippedList=list(zipedFreq)
    termsFreqPerClass.append([currClass]+zippedList)
    # aspect 2
    print ("Class",currClass)
    word_frequency_per_class = pd.Series()
    for headline_column in combinedDataDF[headlines]:
        word_frequency_per_headline_column = pd.Series(" ".join(class_df[headline_column]).split()).value_counts()[:12]
        word_frequency_per_class = word_frequency_per_class.append(word_frequency_per_headline_column)
    word_frequency_per_class = word_frequency_per_class.groupby(by=word_frequency_per_class.index).sum().sort_values(ascending=False)
    print(word_frequency_per_class.to_frame('Word Frequency'))

print ("Terms frequency - for each class, number of records with term:")
pd.DataFrame(termsFreqPerClass,columns=['Class','term 1','term 2','term 3','term 4','term 5','term 6','term 7','term 8','term 9','term 10', 'term 11', 'term 12'])


Terms frequency - for each class, number of appearances of term:


Class 0


          Word Frequency
say                 1560
us                  1365
new                 1131
world               1122
kill                1114
israel               877
year                 823
china                813
govern               660
peopl                566
polic                526
russia               364
war                  326
countri              278
protest              213
attack               193
state                191
report               147
u.s.                 108
iran                 106
ban                  102
isra                  95
russian               70
call                  70
one                   69
law                   68
first                 41
bank                  39
uk                    37
use                   35
amp                   35
forc                  33
time                  32
presid                30
militari              29


Class 1


         Word Frequency
say                1809
us                 1510
kill               1375
world              1223
new                1214
israel             1058
china              1056
year                957
govern              885
polic               614
peopl               432
attack              428
russia              360
protest             338
state               244
report              243
war                 220
countri             171
use                 142
call                134
ban                 102
iran                 86
one                  85
uk                   84
presid               81
isra                 80
drug                 53
first                43
forc                 42
death                41
japan                39
time                 38
Terms frequency - for each class, number of records with term:


,Class,term 1,term 2,term 3,term 4,term 5,term 6,term 7,term 8,term 9,term 10,term 11,term 12
0,0,"(say, 748)","(us, 680)","(new, 650)","(world, 638)","(kill, 630)","(year, 583)","(govern, 572)","(china, 561)","(peopl, 549)","(report, 497)","(polic, 492)","(countri, 490)"
1,1,"(say, 836)","(us, 759)","(kill, 735)","(world, 706)","(new, 701)","(year, 676)","(china, 656)","(govern, 645)","(polic, 596)","(state, 591)","(israel, 591)","(peopl, 582)"


##Question 2: machine learning classifier

In [69]:
from sklearn.model_selection import train_test_split
# x = combined_news_djia_df[combined_news_djia_df.columns[2:4]]
x=combinedDataDF.drop(['Label'], axis=1)
y = combinedDataDF[["Label"]]

Split the data into train and test (30%-Test, 70%-Train)

In [70]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.3, random_state=0)

In [77]:
# combine all text column, and convert to list
xTrain = xTrain.apply(lambda x: " ".join(x), axis=1)
xTest = xTest.apply(lambda x: " ".join(x), axis=1)
xTrain=xTrain.tolist()
xTest=xTest.tolist()
yTrain=yTrain['Label'].tolist()
yTest=yTest['Label'].tolist()

AttributeError: 'list' object has no attribute 'apply'

In [72]:
import numpy as np
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron


The first model we will use is SVM (not Deep Learning, aka Keras)

In [73]:
def classify(extractM,machineLearningM, parameters): 
            pipeline = Pipeline([('vect', extractM), ('clf', machineLearningM)])
            gs_clf = GridSearchCV(pipeline, parameters, n_jobs=1)
            gs_clf = gs_clf.fit(xTrain, yTrain)
            prediction = gs_clf.predict(xTest)
            accuracy = metrics.accuracy_score(yTest, prediction)
            # print(accuracy)
            return [accuracy, gs_clf.best_params_]

In [74]:
results=[]
parameters = {'vect__max_df': (0.3,0.4,0.5,0.6,0.7,0.75, 1.0),'clf__alpha': (0.0001, 0.01,1.0)}
#TF-IDF & SVM
ans = classify(TfidfVectorizer(sublinear_tf=True,stop_words='english'),SGDClassifier() ,parameters)
accuracy = ans[0]
params = ans[1]
results.append(['TF-IDF' , 'SVM', params,accuracy])

In [75]:
print(results)

[['TF-IDF', 'SVM', {'clf__alpha': 0.01, 'vect__max_df': 0.3}, 0.5293132328308208]]


The second model we will use is Neural Network (Deep Learning)

In [76]:
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


ImportError: DLL load failed: The specified procedure could not be found.

In [1]:
#create the model
top_words=10000
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
tfIdfVectorizer=TfidfVectorizer(sublinear_tf=True,stop_words='english')
xTrain2=tfIdfVectorizer.fit_transform(xTrain)
xTest2=tfIdfVectorizer.fit_transform(xTest)
model.fit(xTrain2, yTrain, validation_data=(xTest2, yTest), epochs=3, batch_size=128)

NameError: name 'Sequential' is not defined

get model accuracy

In [20]:
scores = model.evaluate(xTest2, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))